In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import POLOCM2
from utils import read_plan
from evaluator import ExecutabilityEvaluator

In [2]:
def test_exe(domain_name, trace, test_trace=None):
    debug = {
        # 'get_transition_sets': True,
        # 'extract_holes': True,
        # 'find_holes': True,
    }
    try:
        polocm2 = POLOCM2(state_param=True, viz=False, debug=debug)
        model, _, _ = polocm2.extract_model([trace])
        
        actions = model.actions
        
        pddl_domain = model.to_pddl_domain("test_bw")
        evaluator = ExecutabilityEvaluator(pddl_domain, 'overall', debug=True)
        if test_trace is not None:
            action_trace = [step.action for step in test_trace]
            exe = evaluator.get_overall_executability('l', action_trace, set(), set())
        else:
            action_trace = [step.action for step in trace]
            exe = evaluator.get_overall_executability('l', action_trace, set(), set())
        if exe < 1:
            print(f"Warning! domain: {domain_name} - executability: {exe}")
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0

In [3]:
def test_acceptance(domain_name,train_traces, test_traces):
    debug = {}
    try:
        polocm2 = POLOCM2(state_param=True, viz=False, debug=debug)
        model,_,_ = polocm2.extract_model(train_traces)
        learned_domain = model.to_pddl_domain(domain_name)

        evaluator = ExecutabilityEvaluator(learned_domain, debug=True)
        res = []
        for trace in test_traces:
            valid_acceptance, _ = evaluator.get_acceptance_rate(trace, trace)
            res.append(valid_acceptance)
        if len(res) == 0:
            return 0
        return sum(res) / len(res)
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,1

In [4]:
test_plan = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
test_trace = read_plan(test_plan)
po_trace = test_trace.to_partial_ordered_trace(0.3)

test_plan2 = "(pickup b2?object),(stack b2?object b1?object),(unstack b3?object b4?object),(putdown b3?object)"
test_trace2 = read_plan(test_plan)
po_trace2 = test_trace.to_partial_ordered_trace(0.3)

# test_exe("plan1", trace)
test_acceptance("plan2", [po_trace, po_trace2], [test_trace])
# plan2 = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
# test_exe("test_bw", read_plan(plan2))

[          pickup.0  stack.0
pickup.0         0        1
stack.0          1        0,           stack.1  pickup.1  stack.2
stack.1         0         0        1
pickup.1        1         0        0
stack.2         0         0        0]
Error processing domain plan2: 'bool' object is not subscriptable


(0, 1)

In [5]:
# with open("../../data/plain_traces/plain_traces.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         details = line.split("&&")
#         name = f"{details[0]}-{details[2]}-{details[3]}"
      
#         plan = details[-1]
#         trace = read_plan(plan)
#         po_trace = trace.to_partial_ordered_trace(0.3)
#         test_exe(name, po_trace)